In [51]:
from dotenv import load_dotenv

load_dotenv()

True

In [52]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", streaming=True)

In [53]:
import fitz  # pymupdf
import os

def extract_text_with_fitz(pdf_path):
    """PyMuPDF로 텍스트 추출 - 한글 처리 개선"""
    try:
        doc = fitz.open(pdf_path)
        print(f"PDF 파일: {pdf_path}")
        print(f"총 페이지 수: {len(doc)}")
        
        full_text = ""
        
        for page_num in range(len(doc)):
            print(f"페이지 {page_num + 1} 처리 중...")
            page = doc[page_num]
            
            # 텍스트 추출
            text = page.get_text()
            
            if text and text.strip():
                # 한글 처리 및 텍스트 정리
                text = text.replace('\x00', '')  # null 문자 제거
                text = text.replace('\ufeff', '')  # BOM 제거
                text = text.replace('\r\n', '\n')  # 줄바꿈 정리
                text = text.replace('\r', '\n')
                
                print(f"  페이지 {page_num + 1}: {len(text)} 글자 추출")
                print(f"  첫 50글자: {text[:50]}")
                
                full_text += text + "\n\n"  # 페이지 구분
            else:
                print(f"  페이지 {page_num + 1}: 텍스트 없음")
        
        doc.close()
        print(f"\n총 추출된 텍스트: {len(full_text)} 글자")
        return full_text
        
    except Exception as e:
        print(f"PyMuPDF 오류: {e}")
        return ""

# 텍스트 추출
pdf_file = "2024-PR-15.pdf"
full_text = extract_text_with_fitz(pdf_file)

if full_text:
    print("✓ 텍스트 추출 성공!")
    print(f"전체 텍스트 첫 200글자:")
    print(full_text[:300])
else:
    print("✗ 텍스트 추출 실패")

PDF 파일: 2024-PR-15.pdf
총 페이지 수: 127
페이지 1 처리 중...
  페이지 1: 61 글자 추출
  첫 50글자: 약자와 동행하는 서울의 교통
이신해   김승준   한영준   양재환 
윤서연   연준형  
페이지 2 처리 중...
  페이지 2: 16 글자 추출
  첫 50글자: 약자와 동행하는 서울의 교통

페이지 3 처리 중...
  페이지 3: 257 글자 추출
  첫 50글자: 이 보고서의 내용은 연구진의 견해로서 
서울특별시의 정책과는 다를 수도 있습니다.
연구책임
페이지 4 처리 중...
  페이지 4: 1010 글자 추출
  첫 50글자: i
약
자
와
 동
행
하
는
 서
울
의
 교
통
서울시, 다양한 교통약자 불편 개선 위
페이지 5 처리 중...
  페이지 5: 979 글자 추출
  첫 50글자: ii
요
약
서울 장시간 통근자 지원 시급…혼잡 완화·서비스 수준 보장 방안 필요
교통 체
페이지 6 처리 중...
  페이지 6: 321 글자 추출
  첫 50글자: iii
약
자
와
 동
행
하
는
 서
울
의
 교
통
목차
01 연구개요
2
1_연구배경
페이지 7 처리 중...
  페이지 7: 649 글자 추출
  첫 50글자: iv
목
차
표목차
[표 2-1] 교통부문에서의 약자 정의와 권리, 책무 
14
[표 3-
페이지 8 처리 중...
  페이지 8: 752 글자 추출
  첫 50글자: v
약
자
와
 동
행
하
는
 서
울
의
 교
통
그림목차
[그림 1-1] 약자의 범위 
페이지 9 처리 중...
  페이지 9: 1125 글자 추출
  첫 50글자: vi
목
차
[그림 3-15] 고령자가 걸어서 동네 외출 시 가장 불편한 점
38
[그림 
페이지 10 처리 중...
  페이지 10: 820 글자 추출
  첫 50글자: vii
약
자
와
 동
행
하
는
 서
울
의
 교
통
[그림 4-24] 현재 거주지 선택
페이지 11 처리 중...
  페이지 11: 32 글자 추출
  첫 50글자: 01
연구개요
1_연구

In [54]:
def chunk_text(text, chunk_size=2000, overlap=100):
    """텍스트를 청크로 나누는 함수"""
    if not text.strip():
        print("빈 텍스트입니다.")
        return []
    
    chunks = []
    start = 0
    
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end].strip()
        
        if chunk:  # 빈 청크가 아닌 경우만 추가
            chunks.append(chunk)
        
        start = end - overlap
    
    return chunks

# 텍스트 청킹
if full_text:
    chunks = chunk_text(full_text)
    
    print(f"\n=== 청킹 결과 ===")
    print(f"총 청크 개수: {len(chunks)}")
    
    if chunks:
        chunk_lengths = [len(chunk) for chunk in chunks]
        print(f"청크 길이 - 최소: {min(chunk_lengths)}, 최대: {max(chunk_lengths)}, 평균: {sum(chunk_lengths)/len(chunk_lengths):.1f}")
        
        print(f"\n첫 번째 청크 미리보기:")
        print(chunks[0][:200] + "..." if len(chunks[0]) > 200 else chunks[0])
        
        print(f"\n마지막 청크 미리보기:")
        print(chunks[-1][:200] + "..." if len(chunks[-1]) > 200 else chunks[-1])


=== 청킹 결과 ===
총 청크 개수: 49
청크 길이 - 최소: 129, 최대: 2000, 평균: 1961.4

첫 번째 청크 미리보기:
약자와 동행하는 서울의 교통
이신해   김승준   한영준   양재환 
윤서연   연준형   김영범   김지한


약자와 동행하는 서울의 교통


이 보고서의 내용은 연구진의 견해로서 
서울특별시의 정책과는 다를 수도 있습니다.
연구책임
이신해 서울연구원 스마트교통연구실 선임연구위원
연구진
김승준 서울연구원 스마트교통연구실 선임연구위원
한영준 서울연구원 스마...

마지막 청크 미리보기:
-15
발행인  오균
발행일  2025년 5월 30일
발행처  서울연구원
ISBN  979-11-5700-908-4 95350  비매품
06756 서울특별시 서초구 남부순환로 340길 57
이 출판물의 판권은 서울연구원에 속합니다.


In [ ]:
import faiss
import numpy as np
from langchain_openai import OpenAIEmbeddings
import pickle
import os
from datetime import datetime

# OpenAI API 키 설정 (환경변수에서 읽어오거나 직접 설정)
# os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY  # 필요한 경우

def save_vectorstore_openai(index, chunks, metadatas, pdf_filename, save_name="pdf_openai_vectors"):
    """벡터 스토어 저장"""
    try:
        # FAISS 인덱스 저장
        faiss.write_index(index, f'{save_name}_vectors.index')
        
        # 청크와 메타데이터 저장
        with open(f'{save_name}_data.pkl', 'wb') as f:
            pickle.dump({
                'chunks': chunks,
                'metadatas': metadatas,
                'pdf_filename': pdf_filename,
                'chunk_count': len(chunks),
                'embedding_model': 'text-embedding-3-large',
                'embedding_type': 'openai'
            }, f)
        
        return True
        
    except Exception as e:
        print(f"저장 오류: {e}")
        return False

def create_faiss_vectorstore_openai(chunks, pdf_filename, save_name="pdf_openai_vectors"):
    """청크들을 FAISS 벡터 스토어로 변환하고 자동 저장 (OpenAI 임베딩 사용)"""
    if not chunks:
        print("청크가 없습니다.")
        return None, None, None, None
    
    print("OpenAI 임베딩 모델 로드 중...")
    embedding = OpenAIEmbeddings(model='text-embedding-3-large')
    
    print(f"임베딩 생성 중... ({len(chunks)}개 청크)")
    print("OpenAI API 호출 중이므로 시간이 걸릴 수 있습니다...")
    
    try:
        # 청크들을 임베딩으로 변환
        embeddings = embedding.embed_documents(chunks)
        embeddings = np.array(embeddings).astype('float32')
        
        print(f"임베딩 완료! 차원: {embeddings.shape[1]}, 개수: {embeddings.shape[0]}")
        
        # FAISS 인덱스 생성
        dimension = embeddings.shape[1]
        index = faiss.IndexFlatL2(dimension)  # L2 거리 기반 인덱스
        index.add(embeddings)
        
        # 메타데이터 생성
        metadatas = []
        for i, chunk in enumerate(chunks):
            metadatas.append({
                'chunk_id': i,
                'source': pdf_filename,
                'chunk_size': len(chunk),
                'preview': chunk[:100] + "..." if len(chunk) > 100 else chunk
            })
        
        print(f"FAISS 벡터 스토어 생성 완료! {index.ntotal}개 벡터 저장됨")
        
        # 자동 저장
        print("벡터 스토어를 자동 저장 중...")
        if save_vectorstore_openai(index, chunks, metadatas, pdf_filename, save_name):
            print("✓ 자동 저장 완료!")
        else:
            print("⚠️  자동 저장 실패")
        
        return index, embedding, chunks, metadatas
        
    except Exception as e:
        print(f"임베딩 생성 오류: {e}")
        print("OpenAI API 키가 설정되어 있는지 확인해주세요.")
        return None, None, None, None

def load_or_create_vectorstore(chunks, pdf_filename, save_name="pdf_openai_vectors"):
    """
    벡터 스토어가 있으면 로드하고, 없으면 새로 생성하는 함수
    """
    index_file = f'{save_name}_vectors.index'
    data_file = f'{save_name}_data.pkl'
    
    # 저장된 파일들이 모두 존재하는지 확인
    if os.path.exists(index_file) and os.path.exists(data_file):
        print("기존 벡터 스토어를 찾았습니다. 로드 중...")
        
        try:
            # 저장된 데이터 로드
            index = faiss.read_index(index_file)
            
            with open(data_file, 'rb') as f:
                data = pickle.load(f)
            
            # 저장된 PDF 파일명과 현재 파일명 비교
            if data['pdf_filename'] == pdf_filename:
                print("✓ 동일한 PDF 파일의 벡터 스토어 로드 성공!")
                
                # OpenAI 임베딩 모델 로드
                embedding_model = OpenAIEmbeddings(model='text-embedding-3-large')
                
                print(f"- 벡터 개수: {index.ntotal}")
                print(f"- 청크 개수: {data['chunk_count']}")
                print(f"- 원본 파일: {data['pdf_filename']}")
                print(f"- 임베딩 모델: {data.get('embedding_model', '정보 없음')}")
                
                return index, embedding_model, data['chunks'], data['metadatas']
            else:
                print(f"⚠️  다른 PDF 파일의 벡터 스토어입니다.")
                print(f"   저장된 파일: {data['pdf_filename']}")
                print(f"   현재 파일: {pdf_filename}")
                print("   새로운 벡터 스토어를 생성합니다...")
                
        except Exception as e:
            print(f"⚠️  벡터 스토어 로드 중 오류 발생: {e}")
            print("   새로운 벡터 스토어를 생성합니다...")
    
    else:
        print("저장된 벡터 스토어를 찾을 수 없습니다. 새로 생성합니다...")
    
    # 새로운 벡터 스토어 생성
    print("\n=== 새로운 벡터 스토어 생성 ===")
    return create_faiss_vectorstore_openai(chunks, pdf_filename, save_name)

# FAISS 벡터 스토어 자동 로드/생성
if 'chunks' in locals() and chunks:
    print("=== 벡터 스토어 처리 ===")
    index, embedding_model, chunks, metadatas = load_or_create_vectorstore(
        chunks, 
        pdf_file, 
        save_name="my_pdf_vectors"  # 원하는 저장 이름으로 변경 가능
    )
    
    if index is not None:
        print("✓ 벡터 스토어 준비 완료!")
        print("이제 검색을 할 수 있습니다!")
    else:
        print("✗ 벡터 스토어 준비 실패")
else:
    print("청크가 없습니다. 먼저 텍스트 추출과 청킹을 실행해주세요.")

=== 벡터 스토어 처리 ===
기존 벡터 스토어를 찾았습니다. 로드 중...
✓ 동일한 PDF 파일의 벡터 스토어 로드 성공!
- 벡터 개수: 49
- 청크 개수: 49
- 원본 파일: 2024-PR-15.pdf
- 임베딩 모델: text-embedding-3-large
✓ 벡터 스토어 준비 완료!
이제 검색을 할 수 있습니다!


In [56]:
# 필요한 라이브러리들
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolExecutor
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from typing import TypedDict, List, Dict, Any
import json

# 이미 가지고 있는 검색 관련 함수들 활용

In [57]:
class AgentState(TypedDict):
    messages: List[Any]
    user_query: str
    search_results: List[Dict]
    summary: str
    tool_calls: List[Dict]

In [58]:
# 새 셀 추가 (cell [28])
import numpy as np

def search_documents_openai(query, index, embedding_model, chunks, metadatas, k=3):
    """
    OpenAI 임베딩을 사용한 문서 검색
    """
    try:
        # 쿼리를 임베딩으로 변환
        query_embedding = embedding_model.embed_query(query)
        query_embedding = np.array([query_embedding]).astype('float32')
        
        # FAISS로 유사한 문서 검색
        distances, indices = index.search(query_embedding, k)
        
        # 결과 포맷팅
        results = []
        for i, (distance, idx) in enumerate(zip(distances[0], indices[0])):
            if idx < len(chunks):  # 유효한 인덱스인지 확인
                results.append({
                    'chunk_id': idx,
                    'content': chunks[idx],
                    'score': 1.0 - (distance / 2.0),  # 거리를 유사도로 변환
                    'metadata': metadatas[idx] if idx < len(metadatas) else {}
                })
        
        return results
        
    except Exception as e:
        print(f"검색 오류: {e}")
        return []

In [59]:
# cell [30] 교체
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
import tempfile
import os

# 웹 검색 도구 초기화
try:
    search_tool = DuckDuckGoSearchRun()
    print("✅ 웹 검색 도구 준비 완료")
except Exception as e:
    print(f"⚠️ 웹 검색 도구 초기화 실패: {e}")
    search_tool = None

@tool
def search_uploaded_documents(query: str, k: int = 3) -> str:
    """
    업로드된 문서에서 내용을 검색합니다.
    """
    global index, embedding_model, chunks, metadatas
    
    if 'index' not in globals() or index is None:
        return "업로드된 문서가 없습니다. 먼저 파일을 업로드해주세요."
    
    try:
        results = search_documents_openai(query, index, embedding_model, chunks, metadatas, k)
        
        if not results:
            return f"'{query}'에 대한 검색 결과가 없습니다."
        
        formatted_results = []
        for i, result in enumerate(results):
            formatted_results.append(
                f"[문서 검색결과 {i+1}]\n"
                f"유사도: {result['score']:.3f}\n"
                f"내용: {result['content'][:400]}{'...' if len(result['content']) > 400 else ''}\n"
            )
        
        return "\n\n".join(formatted_results)
        
    except Exception as e:
        return f"문서 검색 중 오류: {str(e)}"

@tool
def web_search(query: str) -> str:
    """
    웹에서 최신 정보를 검색합니다.
    """
    if search_tool is None:
        return "웹 검색 기능을 사용할 수 없습니다."
    
    try:
        results = search_tool.run(query)
        return f"🌐 웹 검색 결과:\n{results}"
    except Exception as e:
        return f"웹 검색 중 오류 발생: {str(e)}"

@tool
def summarize_content(content: str, style: str = "general") -> str:
    """
    주어진 내용을 요약합니다.
    """
    if not content.strip():
        return "요약할 내용이 없습니다."
    
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)
    
    style_prompts = {
        "general": "다음 내용을 자연스럽게 요약해주세요:",
        "bullet_points": "다음 내용을 주요 포인트별로 불릿 포인트로 요약해주세요:",
        "executive": "다음 내용을 경영진을 위한 간단한 요약으로 작성해주세요:",
        "detailed": "다음 내용을 상세하게 요약하되 중요한 내용은 빠뜨리지 마세요:"
    }
    
    prompt = style_prompts.get(style, style_prompts["general"])
    
    try:
        response = llm.invoke([
            HumanMessage(content=f"{prompt}\n\n{content}")
        ])
        return response.content
    except Exception as e:
        return f"요약 중 오류 발생: {str(e)}"

@tool
def get_document_info() -> str:
    """
    현재 로드된 PDF 문서의 정보를 반환합니다.
    """
    global chunks, metadatas, pdf_file
    
    if 'chunks' not in globals() or not chunks:
        return "로드된 문서가 없습니다."
    
    total_chars = sum(len(chunk) for chunk in chunks)
    
    return f"""
📄 현재 로드된 문서 정보:
- 파일명: {pdf_file if 'pdf_file' in globals() else '알 수 없음'}
- 총 청크 수: {len(chunks)}
- 총 글자 수: {total_chars:,}
- 평균 청크 크기: {total_chars // len(chunks):,} 글자

🛠️ 사용 가능한 기능:
- 문서 검색: "교통약자에 대해 찾아줘"
- 웹 검색: "최신 AI 뉴스 검색해줘"  
- 내용 요약: "이 문서를 요약해줘"
"""

✅ 웹 검색 도구 준비 완료


In [ ]:
# 새 셀 추가
from langgraph.prebuilt import ToolNode

def enhanced_call_model(state: AgentState) -> AgentState:
    """향상된 LLM 모델 호출"""
    messages = state["messages"]
    
    system_prompt = """
당신은 다기능 AI 어시스턴트입니다. 다음 기능들을 제공할 수 있습니다:

🔍 **검색 기능**:
- search_uploaded_documents: 업로드된 PDF 문서 검색
- web_search: 최신 웹 정보 검색

📝 **분석 기능**:
- summarize_content: 다양한 스타일로 내용 요약
- get_document_info: 현재 문서 상태 확인

사용자 요청에 따라 적절한 도구를 선택하세요:

1. 업로드된 문서에 대한 질문 → search_uploaded_documents 사용
2. 최신 정보나 일반 질문 → web_search 사용  
3. 요약 요청 → summarize_content 사용
4. 문서 정보 요청 → get_document_info 사용

항상 한국어로 친절하고 정확하게 답변해주세요.
"""
    
    if not messages or "system" not in str(messages[0]):
        messages = [HumanMessage(content=system_prompt)] + messages
    
    # 모든 도구 바인딩
    llm = ChatOpenAI(model="gpt-4o", temperature=0.1)
    tools = [search_uploaded_documents, web_search, summarize_content, get_document_info]
    llm_with_tools = llm.bind_tools(tools)
    
    response = llm_with_tools.invoke(messages)
    
    return {"messages": messages + [response]}

# 새로운 도구 노드
tools = [search_uploaded_documents, web_search, summarize_content, get_document_info]
enhanced_tool_node = ToolNode(tools)

In [61]:
# @tool
# def summarize_content(content: str, focus: str = "general") -> str:
#     """
#     주어진 내용을 요약합니다.
    
#     Args:
#         content: 요약할 내용
#         focus: 요약 초점 ("general", "key_points", "technical", "brief")
    
#     Returns:
#         요약된 내용
#     """
#     if not content.strip():
#         return "요약할 내용이 없습니다."
    
#     # OpenAI LLM 모델 사용
#     llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)
    
#     focus_prompts = {
#         "general": "다음 내용을 전반적으로 요약해주세요:",
#         "key_points": "다음 내용에서 핵심 포인트들만 정리해주세요:",
#         "technical": "다음 내용에서 기술적인 부분을 중심으로 요약해주세요:",
#         "brief": "다음 내용을 간단히 한 문단으로 요약해주세요:"
#     }
    
#     prompt = focus_prompts.get(focus, focus_prompts["general"])
    
#     try:
#         response = llm.invoke([
#             HumanMessage(content=f"{prompt}\n\n{content}")
#         ])
#         return response.content
#     except Exception as e:
#         return f"요약 중 오류 발생: {str(e)}"

In [62]:
@tool
def get_document_info() -> str:
    """
    현재 로드된 PDF 문서의 정보를 반환합니다.
    
    Returns:
        문서 정보
    """
    global chunks, metadatas, pdf_file
    
    if 'chunks' not in globals() or not chunks:
        return "로드된 문서가 없습니다."
    
    total_chars = sum(len(chunk) for chunk in chunks)
    
    return f"""
현재 로드된 문서 정보:
- 파일명: {pdf_file if 'pdf_file' in globals() else '알 수 없음'}
- 총 청크 수: {len(chunks)}
- 총 글자 수: {total_chars:,}
- 평균 청크 크기: {total_chars // len(chunks):,} 글자
"""

In [63]:
def should_continue(state: AgentState) -> str:
    """다음 단계를 결정하는 함수"""
    messages = state["messages"]
    last_message = messages[-1]
    
    # 도구 호출이 있으면 도구 실행
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        return "tools"
    
    # 그렇지 않으면 종료
    return END

def call_model(state: AgentState) -> AgentState:
    """LLM 모델 호출"""
    messages = state["messages"]
    
    # 시스템 프롬프트
    system_prompt = """
당신은 PDF 문서 검색 및 요약 전문 에이전트입니다.

사용 가능한 도구들:
1. search_pdf_documents: PDF에서 관련 내용 검색
2. summarize_content: 내용 요약
3. get_document_info: 문서 정보 조회

사용자의 질문에 따라 적절한 도구를 사용하여 답변하세요:
- 특정 내용을 찾고 싶다면 search_pdf_documents를 사용
- 검색된 내용을 요약하고 싶다면 summarize_content를 사용
- 문서 정보가 궁금하다면 get_document_info를 사용

항상 한국어로 친절하고 정확하게 답변해주세요.
"""
    
    # 시스템 메시지가 없다면 추가
    if not messages or not any(isinstance(msg, HumanMessage) for msg in messages[:1]):
        messages = [HumanMessage(content=system_prompt)] + messages
    
    # LLM 호출
    llm = ChatOpenAI(model="gpt-4o", temperature=0.1)
    llm_with_tools = llm.bind_tools([search_pdf_documents, summarize_content, get_document_info])
    
    response = llm_with_tools.invoke(messages)
    
    return {"messages": messages + [response]}

def call_tools(state: AgentState) -> AgentState:
    """도구 실행"""
    messages = state["messages"]
    last_message = messages[-1]
    
    # 도구 실행기 생성
    tools = [search_pdf_documents, summarize_content, get_document_info]
    tool_executor = ToolExecutor(tools)
    
    tool_messages = []
    
    # 각 도구 호출 실행
    for tool_call in last_message.tool_calls:
        try:
            result = tool_executor.invoke(tool_call)
            tool_messages.append(
                ToolMessage(
                    content=str(result),
                    tool_call_id=tool_call["id"]
                )
            )
        except Exception as e:
            tool_messages.append(
                ToolMessage(
                    content=f"도구 실행 오류: {str(e)}",
                    tool_call_id=tool_call["id"]
                )
            )
    
    return {"messages": messages + tool_messages}

In [64]:
# cell [36] 교체
# 향상된 그래프 생성
enhanced_graph = StateGraph(AgentState)

enhanced_graph.add_node("agent", enhanced_call_model)
enhanced_graph.add_node("tools", enhanced_tool_node)

enhanced_graph.set_entry_point("agent")
enhanced_graph.add_conditional_edges(
    "agent",
    should_continue,
    {
        "tools": "tools",
        END: END
    }
)
enhanced_graph.add_edge("tools", "agent")

# 그래프 컴파일
graph = enhanced_graph.compile()

print("✅ 향상된 멀티모달 에이전트 준비 완료!")
print("🔍 웹 검색 + 📄 문서 검색 + 📝 요약 기능 모두 사용 가능")

✅ 향상된 멀티모달 에이전트 준비 완료!
🔍 웹 검색 + 📄 문서 검색 + 📝 요약 기능 모두 사용 가능


In [65]:
# # def create_pdf_agent():
#     # """PDF 검색 요약 에이전트 생성"""
    
#     # 그래프 생성
# graph_builder = StateGraph(AgentState)

# # 노드 추가
# graph_builder.add_node("agent", call_model)
# graph_builder.add_node("tools", call_tools)

# # 시작점 설정
# graph_builder.set_entry_point("agent")

# # 조건부 엣지 추가
# graph_builder.add_conditional_edges(
#     "agent",
#     should_continue,
#     {
#         "tools": "tools",
#         END: END
#     }
# )

# # 도구에서 다시 에이전트로
# graph_builder.add_edge("tools", "agent")

# # 그래프 컴파일
# graph = graph_builder.compile()

# graph

# # 에이전트 생성
# # pdf_agent = create_pdf_agent()


In [66]:
# cell [55] 수정된 버전
def run_enhanced_agent(user_query: str) -> str:
    """향상된 에이전트 실행"""
    
    initial_state = {
        "messages": [HumanMessage(content=user_query)],
        "user_query": user_query,
        "search_results": [],
        "summary": "",
        "tool_calls": []
    }
    
    try:
        result = graph.invoke(initial_state)  # graph 사용 (이미 정의됨)
        final_message = result["messages"][-1]
        
        if hasattr(final_message, 'content'):
            return final_message.content
        else:
            return str(final_message)
            
    except Exception as e:
        return f"에이전트 실행 중 오류 발생: {str(e)}"

# pdf_agent는 graph로 대체
pdf_agent = graph

def interactive_pdf_agent():
    """대화형 PDF 에이전트"""
    print("\n" + "="*60)
    print("🤖 멀티모달 AI 어시스턴트")
    print("="*60)
    print("• '종료', 'quit', 'exit'를 입력하면 종료됩니다")
    print("• 예시 질문:")
    print("  - '교통약자에 대해 검색해줘'")
    print("  - '이 문서를 요약해줘'")
    print("  - '최신 AI 뉴스 검색해줘'")
    print("  - '문서 정보를 알려줘'")
    print("-"*60)
    
    while True:
        try:
            user_input = input("\n💬 질문을 입력하세요: ").strip()
            
            if user_input.lower() in ['종료', 'quit', 'exit', 'q']:
                print("에이전트를 종료합니다. 👋")
                break
            
            if not user_input:
                continue
            
            print("\n🤖 에이전트가 작업 중입니다...")
            print("-" * 40)
            
            # 향상된 에이전트 실행
            response = run_enhanced_agent(user_input)
            
            print(f"\n🎯 답변:\n{response}")
            print("-" * 60)
            
        except KeyboardInterrupt:
            print("\n\n에이전트를 종료합니다. 👋")
            break
        except Exception as e:
            print(f"오류 발생: {e}")

In [67]:
# cell [56] 수정된 버전
# 벡터 스토어가 준비되었는지 확인
if 'index' in locals() and index is not None:
    print("✅ 멀티모달 AI 어시스턴트가 준비되었습니다!")
    
    # 테스트 실행
    test_query = "문서 정보를 알려줘"
    print(f"\n🧪 테스트 실행: {test_query}")
    result = run_enhanced_agent(test_query)
    print(f"결과: {result[:200]}...")
    
    # 대화형 실행
    print("\n🚀 대화형 에이전트를 시작하시겠습니까? (y/n)")
    if input("입력: ").lower() in ['y', 'yes', '네', 'ㅇ']:
        interactive_pdf_agent()
        
else:
    print("❌ 벡터 스토어가 준비되지 않았습니다.")
    print("먼저 PDF 처리와 벡터 스토어 생성을 완료해주세요.")

✅ 멀티모달 AI 어시스턴트가 준비되었습니다!

🧪 테스트 실행: 문서 정보를 알려줘
결과: 에이전트 실행 중 오류 발생: Error code: 400 - {'error': {'message': "Invalid parameter: messages with role 'tool' must be a response to a preceeding message with 'tool_calls'.", 'type': 'invalid_request_error', ...

🚀 대화형 에이전트를 시작하시겠습니까? (y/n)

🤖 멀티모달 AI 어시스턴트
• '종료', 'quit', 'exit'를 입력하면 종료됩니다
• 예시 질문:
  - '교통약자에 대해 검색해줘'
  - '이 문서를 요약해줘'
  - '최신 AI 뉴스 검색해줘'
  - '문서 정보를 알려줘'
------------------------------------------------------------

🤖 에이전트가 작업 중입니다...
----------------------------------------

🎯 답변:
안녕하세요! 무엇을 도와드릴까요? 필요한 정보나 도움이 필요하시면 언제든지 말씀해 주세요.
------------------------------------------------------------

🤖 에이전트가 작업 중입니다...
----------------------------------------


c:\AI_Prompt\workspace\ai_agent_work2\langgraph_uv\.venv\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:64: UserWarning: 'api' backend is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(



🎯 답변:
에이전트 실행 중 오류 발생: Error code: 400 - {'error': {'message': "Invalid parameter: messages with role 'tool' must be a response to a preceeding message with 'tool_calls'.", 'type': 'invalid_request_error', 'param': 'messages.[1].role', 'code': None}}
------------------------------------------------------------
에이전트를 종료합니다. 👋


In [68]:
# # 벡터 스토어가 준비되었는지 확인
# if 'index' in locals() and index is not None:
#     print("✅ PDF 검색 요약 에이전트가 준비되었습니다!")
    
#     # 테스트 실행
#     test_query = "문서 정보를 알려줘"
#     print(f"\n🧪 테스트 실행: {test_query}")
#     result = run_enhanced_agent(test_query)
#     print(f"결과: {result}")
    
#     # 대화형 실행
#     print("\n🚀 대화형 에이전트를 시작하시겠습니까? (y/n)")
#     if input("입력: ").lower() in ['y', 'yes', '네', 'ㅇ']:
#         interactive_pdf_agent()
        
# else:
#     print("❌ 벡터 스토어가 준비되지 않았습니다.")
#     print("먼저 PDF 처리와 벡터 스토어 생성을 완료해주세요.")

In [ ]:
# %pip install streamlit

Note: you may need to restart the kernel to use updated packages.


c:\AI_Prompt\workspace\ai_agent_work2\langgraph_uv\.venv\Scripts\python.exe: No module named pip


In [70]:
def run_safe_agent(user_query: str) -> str:
    """안전한 에이전트 실행 - 에러 방지"""
    
    try:
        # 간단한 직접 응답 케이스들
        if "안녕" in user_query or "hello" in user_query.lower():
            return "안녕하세요! 저는 멀티모달 AI 어시스턴트입니다. 문서 검색, 웹 검색, 요약 등을 도와드릴 수 있어요. 무엇을 도와드릴까요?"
        
        # 도구가 필요한 경우들
        if any(keyword in user_query for keyword in ["문서", "정보", "알려", "검색", "요약", "찾아"]):
            
            # 문서 정보 요청
            if "문서" in user_query and "정보" in user_query:
                return get_document_info().invoke({})
            
            # 문서 검색 요청  
            elif any(keyword in user_query for keyword in ["교통", "고령", "약자", "정책"]):
                if 'index' in globals() and index is not None:
                    return search_uploaded_documents.invoke({"query": user_query})
                else:
                    return "문서가 업로드되지 않았습니다. 먼저 PDF 파일을 업로드해주세요."
            
            # 웹 검색 요청
            elif any(keyword in user_query for keyword in ["최신", "뉴스", "날씨", "AI"]):
                return web_search.invoke({"query": user_query})
            
            # 요약 요청
            elif "요약" in user_query:
                if 'chunks' in globals() and chunks:
                    # 전체 문서의 일부를 요약
                    sample_content = "\n".join(chunks[:3])  # 처음 3개 청크
                    return summarize_content.invoke({"content": sample_content})
                else:
                    return "요약할 문서가 없습니다. 먼저 PDF 파일을 업로드해주세요."
        
        # 기본 LLM 응답
        llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
        response = llm.invoke([HumanMessage(content=user_query)])
        return response.content
        
    except Exception as e:
        return f"죄송합니다. 처리 중 오류가 발생했습니다: {str(e)}\n\n다른 질문을 시도해보시거나, 문서를 먼저 업로드해주세요."

In [71]:
# import gradio as gr
# import traceback

# def simple_file_handler(file_obj):
#     """파일 업로드 테스트"""
#     try:
#         if not file_obj:
#             return "파일이 업로드되지 않았습니다."
        
#         file_path = file_obj.name if hasattr(file_obj, 'name') else str(file_obj)
#         return f"✅ 파일 업로드 성공: {file_path}"
        
#     except Exception as e:
#         return f"❌ 파일 처리 오류: {str(e)}\n{traceback.format_exc()}"

# def simple_chat_handler(message, history):
#     """채팅 테스트"""
#     try:
#         if not message.strip():
#             return history, ""
        
#         print(f"받은 메시지: {message}")  # 콘솔에 출력
        
#         # 간단한 응답들
#         if "안녕" in message:
#             response = "안녕하세요! 테스트 중입니다."
#         elif "테스트" in message:
#             response = "테스트가 정상적으로 작동하고 있습니다!"
#         elif "에러" in message:
#             response = "에러 테스트입니다. 이 메시지가 보이면 기본 기능은 작동합니다."
#         else:
#             response = f"받은 메시지: '{message}' - 기본 응답입니다."
        
#         history.append([message, response])
#         return history, ""
        
#     except Exception as e:
#         error_msg = f"❌ 채팅 처리 오류: {str(e)}\n{traceback.format_exc()}"
#         print(error_msg)  # 콘솔에 출력
#         history.append([message, error_msg])
#         return history, ""

# # 간단한 Gradio 인터페이스
# with gr.Blocks(title="🐛 디버깅 테스트") as debug_demo:
    
#     gr.Markdown("# 🐛 디버깅 테스트")
#     gr.Markdown("기본 기능이 작동하는지 확인합니다.")
    
#     with gr.Row():
#         with gr.Column():
#             gr.Markdown("### 📁 파일 테스트")
#             file_upload = gr.File(label="파일 선택", type="file")
#             file_status = gr.Textbox(label="파일 상태", lines=3)
            
#         with gr.Column():
#             gr.Markdown("### 💬 채팅 테스트")
#             chatbot = gr.Chatbot(height=300)
            
#             with gr.Row():
#                 msg_input = gr.Textbox(placeholder="메시지 입력", scale=4)
#                 send_btn = gr.Button("전송", scale=1)
            
#             with gr.Row():
#                 test_btns = [
#                     gr.Button("안녕하세요", size="sm"),
#                     gr.Button("테스트", size="sm"),
#                     gr.Button("에러 테스트", size="sm")
#                 ]
    
#     # 이벤트 연결
#     file_upload.change(simple_file_handler, [file_upload], [file_status])
    
#     msg_input.submit(simple_chat_handler, [msg_input, chatbot], [chatbot, msg_input])
#     send_btn.click(simple_chat_handler, [msg_input, chatbot], [chatbot, msg_input])
    
#     # 테스트 버튼들
#     for i, btn in enumerate(test_btns):
#         test_messages = ["안녕하세요", "테스트", "에러 테스트"]
#         btn.click(
#             lambda hist, msg=test_messages[i]: simple_chat_handler(msg, hist),
#             [chatbot], 
#             [chatbot, msg_input]
#         )

# print("🐛 디버깅 테스트 인터페이스 시작...")
# debug_demo.launch(share=False, server_port=7862)

In [72]:
# import gradio as gr
# from langchain_core.messages import HumanMessage, AIMessage
# import tempfile
# import os
# import shutil

# def process_uploaded_file(file_obj):
#     """업로드된 파일 처리 - 개선된 버전"""
#     if not file_obj:
#         return "파일이 업로드되지 않았습니다."
    
#     try:
#         global full_text, chunks, index, embedding_model, metadatas, pdf_file
        
#         # 파일 객체에서 경로 추출
#         if hasattr(file_obj, 'name'):
#             file_path = file_obj.name
#         elif isinstance(file_obj, str):
#             file_path = file_obj
#         else:
#             return "파일 경로를 확인할 수 없습니다."
        
#         print(f"처리할 파일: {file_path}")  # 디버깅용
        
#         # 파일 존재 확인
#         if not os.path.exists(file_path):
#             return f"파일을 찾을 수 없습니다: {file_path}"
        
#         # PDF 파일인지 확인
#         if not file_path.lower().endswith('.pdf'):
#             return "❌ PDF 파일만 지원됩니다."
        
#         # PDF 텍스트 추출
#         print("PDF 텍스트 추출 중...")
#         full_text = extract_text_with_fitz(file_path)
        
#         if not full_text or len(full_text.strip()) < 10:
#             return "❌ 파일에서 텍스트를 추출할 수 없었습니다. 스캔된 이미지 PDF일 수 있습니다."
        
#         print(f"추출된 텍스트 길이: {len(full_text)}")
        
#         # 텍스트 청킹
#         print("텍스트 청킹 중...")
#         chunks = chunk_text(full_text)
        
#         if not chunks:
#             return "❌ 텍스트 청킹에 실패했습니다."
        
#         print(f"생성된 청크 수: {len(chunks)}")
        
#         # 벡터 스토어 생성
#         print("벡터 스토어 생성 중...")
#         pdf_file = file_path
        
#         index, embedding_model, chunks, metadatas = load_or_create_vectorstore(
#             chunks, 
#             os.path.basename(file_path),  # 파일명만 사용
#             save_name="gradio_uploaded_vectors"
#         )
        
#         if index is None:
#             return "❌ 벡터 스토어 생성에 실패했습니다. OpenAI API 키를 확인해주세요."
        
#         return f"""✅ 파일 처리 완료!
# 📁 파일: {os.path.basename(file_path)}
# 📄 청크 수: {len(chunks)}
# 📝 총 글자 수: {len(full_text):,}
# 🚀 이제 문서에 대해 질문할 수 있습니다!"""
        
#     except Exception as e:
#         print(f"파일 처리 오류: {str(e)}")  # 디버깅용
#         return f"❌ 파일 처리 중 오류 발생: {str(e)}"

# def safe_chat_with_agent(message, history):
#     """안전한 채팅 함수"""
#     try:
#         if not message.strip():
#             return history, ""
        
#         print(f"사용자 질문: {message}")  # 디버깅용
        
#         # 간단한 응답들
#         if any(keyword in message.lower() for keyword in ["안녕", "hello", "hi"]):
#             response = "안녕하세요! 저는 AI 어시스턴트입니다. 업로드하신 문서에 대해 질문해보세요!"
#             history.append([message, response])
#             return history, ""
        
#         # 문서가 로드되었는지 확인
#         if 'chunks' not in globals() or not chunks:
#             response = "먼저 PDF 파일을 업로드해주세요. 업로드가 완료되면 문서 내용에 대해 질문할 수 있습니다."
#             history.append([message, response])
#             return history, ""
        
#         # 문서 정보 요청
#         if any(keyword in message for keyword in ["정보", "문서", "상태"]):
#             response = get_document_info().invoke({})
#             history.append([message, response])
#             return history, ""
        
#         # 문서 검색
#         if any(keyword in message for keyword in ["검색", "찾", "교통", "정책", "고령", "약자"]):
#             try:
#                 search_result = search_uploaded_documents.invoke({"query": message, "k": 3})
#                 response = f"🔍 검색 결과:\n\n{search_result}"
#                 history.append([message, response])
#                 return history, ""
#             except Exception as e:
#                 response = f"검색 중 오류가 발생했습니다: {str(e)}"
#                 history.append([message, response])
#                 return history, ""
        
#         # 요약 요청
#         if "요약" in message:
#             try:
#                 if chunks:
#                     # 처음 몇 개 청크를 요약
#                     sample_content = "\n\n".join(chunks[:3])
#                     summary_result = summarize_content.invoke({"content": sample_content, "style": "general"})
#                     response = f"📝 문서 요약:\n\n{summary_result}"
#                     history.append([message, response])
#                     return history, ""
#                 else:
#                     response = "요약할 문서가 없습니다. 먼저 PDF를 업로드해주세요."
#                     history.append([message, response])
#                     return history, ""
#             except Exception as e:
#                 response = f"요약 중 오류가 발생했습니다: {str(e)}"
#                 history.append([message, response])
#                 return history, ""
        
#         # 웹 검색
#         if any(keyword in message for keyword in ["최신", "뉴스", "웹", "인터넷"]):
#             try:
#                 web_result = web_search.invoke({"query": message})
#                 response = f"🌐 웹 검색 결과:\n\n{web_result}"
#                 history.append([message, response])
#                 return history, ""
#             except Exception as e:
#                 response = f"웹 검색 중 오류가 발생했습니다: {str(e)}"
#                 history.append([message, response])
#                 return history, ""
        
#         # 기본 LLM 응답
#         try:
#             from langchain_openai import ChatOpenAI
#             llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
            
#             # 문서 컨텍스트가 있다면 포함
#             context = ""
#             if 'chunks' in globals() and chunks:
#                 context = f"\n\n참고 문서 정보: 업로드된 문서에는 {len(chunks)}개의 섹션이 있습니다."
            
#             response_obj = llm.invoke([HumanMessage(content=f"{message}{context}")])
#             response = response_obj.content
            
#             history.append([message, response])
#             return history, ""
            
#         except Exception as e:
#             response = f"응답 생성 중 오류가 발생했습니다: {str(e)}"
#             history.append([message, response])
#             return history, ""
        
#     except Exception as e:
#         print(f"채팅 오류: {str(e)}")  # 디버깅용
#         error_response = f"죄송합니다. 처리 중 오류가 발생했습니다: {str(e)}"
#         history.append([message, error_response])
#         return history, ""

# # 수정된 Gradio 인터페이스
# with gr.Blocks(title="🤖 AI 문서 어시스턴트", theme="soft") as demo:
    
#     gr.Markdown("""
#     # 🤖 AI 문서 어시스턴트
    
#     **기능:**
#     - 📁 PDF 파일 업로드 및 분석
#     - 🔍 문서 내용 검색
#     - 📝 문서 요약
#     - 🌐 웹 검색
#     - 💬 자연어 대화
#     """)
    
#     with gr.Row():
#         # 좌측: 파일 업로드
#         with gr.Column(scale=1):
#             gr.Markdown("### 📁 파일 업로드")
            
#             file_upload = gr.File(
#                 label="PDF 파일 선택",
#                 file_types=[".pdf"],
#                 type="file"  # filepath -> file로 변경
#             )
            
#             file_status = gr.Textbox(
#                 label="파일 처리 상태",
#                 value="PDF 파일을 업로드해주세요...",
#                 interactive=False,
#                 lines=4
#             )
            
#             gr.Markdown("""
#             ### 💡 사용 팁
#             **예시 질문:**
#             - "이 문서를 요약해줘"
#             - "교통약자에 대해 검색해줘"  
#             - "문서 정보 알려줘"
#             - "최신 뉴스 검색해줘"
#             """)
        
#         # 우측: 채팅
#         with gr.Column(scale=2):
#             gr.Markdown("### 💬 AI 어시스턴트")
            
#             chatbot = gr.Chatbot(
#                 value=[["시스템", "안녕하세요! PDF 파일을 업로드하고 질문해보세요."]],
#                 height=400,
#                 show_label=False
#             )
            
#             with gr.Row():
#                 msg_input = gr.Textbox(
#                     placeholder="질문을 입력하세요...",
#                     show_label=False,
#                     scale=4
#                 )
#                 send_btn = gr.Button("전송", variant="primary", scale=1)
            
#             with gr.Row():
#                 gr.Button("📄 문서 정보", size="sm"),
#                 gr.Button("📝 문서 요약", size="sm"),
#                 gr.Button("🔍 내용 검색", size="sm"),
#                 gr.Button("🌐 웹 검색", size="sm")
    
#     # 이벤트 핸들러
    
#     # 파일 업로드 처리
#     file_upload.change(
#         process_uploaded_file,
#         inputs=[file_upload],
#         outputs=[file_status]
#     )
    
#     # 채팅 처리
#     msg_input.submit(
#         safe_chat_with_agent,
#         inputs=[msg_input, chatbot],
#         outputs=[chatbot, msg_input]
#     )
    
#     send_btn.click(
#         safe_chat_with_agent,
#         inputs=[msg_input, chatbot],
#         outputs=[chatbot, msg_input]
#     )

# # 실행
# print("🚀 AI 문서 어시스턴트 시작!")
# print("🏠 로컬 주소로 실행합니다...")
# demo.launch(share=False, server_port=7861, debug=True)

In [ ]:
import gradio as gr
import traceback

# 매우 간단한 테스트 버전
def test_chat(message, history):
    """기본 채팅 테스트"""
    try:
        print(f"[DEBUG] 받은 메시지: {message}")
        
        if not message:
            return history, ""
        
        if "안녕" in message:
            response = "안녕하세요! 테스트가 정상 작동합니다."
        elif "파일" in message:
            response = "파일 기능은 준비 중입니다."
        else:
            response = f"메시지를 받았습니다: {message}"
        
        print(f"[DEBUG] 응답: {response}")
        history.append([message, response])
        return history, ""
        
    except Exception as e:
        print(f"[ERROR] {traceback.format_exc()}")
        history.append([message, f"오류 발생: {str(e)}"])
        return history, ""

def test_file_upload(file):
    """파일 업로드 테스트"""
    try:
        print(f"[DEBUG] 파일 업로드: {file}")
        
        if not file:
            return "파일이 없습니다."
        
        file_info = f"파일 정보: {file.name if hasattr(file, 'name') else str(file)}"
        print(f"[DEBUG] {file_info}")
        return f"✅ {file_info}"
        
    except Exception as e:
        print(f"[ERROR] 파일 오류: {traceback.format_exc()}")
        return f"파일 오류: {str(e)}"

# 매우 간단한 인터페이스
with gr.Blocks(title="🧪 기본 테스트") as test_demo:
    gr.Markdown("# 🧪 기본 기능 테스트")
    
    with gr.Row():
        with gr.Column():
            gr.Markdown("### 파일 테스트")
            file_input = gr.File(label="파일 선택")
            file_output = gr.Textbox(label="파일 상태")
            
        with gr.Column():
            gr.Markdown("### 채팅 테스트")
            chatbot = gr.Chatbot(height=300)
            msg_input = gr.Textbox(placeholder="테스트 메시지")
            send_btn = gr.Button("전송")
    
    # 이벤트
    file_input.change(test_file_upload, file_input, file_output)
    msg_input.submit(test_chat, [msg_input, chatbot], [chatbot, msg_input])
    send_btn.click(test_chat, [msg_input, chatbot], [chatbot, msg_input])

if __name__ == "__main__":
    print("🧪 기본 테스트 시작...")
    test_demo.launch(server_port=7865, debug=True, show_error=True)

c:\AI_Prompt\workspace\ai_agent_work2\langgraph_uv\.venv\Lib\site-packages\gradio\deprecation.py:40: UserWarning: `height` is deprecated in `Interface()`, please use it within `launch()` instead.
  warnings.warn(value)


🧪 기본 테스트 시작...
Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


In [ ]:
# 필요한 변수들이 정의되어 있는지 확인
required_functions = [
    'extract_text_with_fitz', 
    'chunk_text', 
    'load_or_create_vectorstore',
    'search_uploaded_documents', 
    'summarize_content', 
    'get_document_info', 
    'web_search'
]

print("=== 함수 정의 상태 확인 ===")
for func in required_functions:
    if func in globals():
        print(f"✅ {func}")
    else:
        print(f"❌ {func} - 정의되지 않음")

# OpenAI API 키 확인
import os
if "OPENAI_API_KEY" in os.environ:
    print("✅ OpenAI API 키 설정됨")
else:
    print("❌ OpenAI API 키 없음")

=== 함수 정의 상태 확인 ===
✅ extract_text_with_fitz
✅ chunk_text
✅ load_or_create_vectorstore
✅ search_uploaded_documents
✅ summarize_content
✅ get_document_info
✅ web_search
✅ OpenAI API 키 설정됨


In [ ]:
# import gradio as gr
# import signal
# import sys

# def safe_launch_demo(demo):
#     """안전한 Gradio 데모 실행"""
    
#     def signal_handler(sig, frame):
#         print('\n\n🛑 프로그램을 안전하게 종료합니다...')
#         sys.exit(0)
    
#     # Ctrl+C 핸들러 등록
#     signal.signal(signal.SIGINT, signal_handler)
    
#     try:
#         print("🚀 AI 문서 어시스턴트 시작!")
#         print("📍 실행 옵션을 선택하세요:")
#         print("1. 로컬에서만 실행 (빠름)")
#         print("2. 공개 링크 생성 (느림, 인터넷 필요)")
#         print("3. 기본 실행")
        
#         choice = input("선택 (1-3, 기본값: 1): ").strip()
        
#         if choice == "2":
#             print("🌐 공개 링크를 생성하는 중입니다... (시간이 걸릴 수 있습니다)")
#             print("⚠️  중단하려면 Ctrl+C를 누르세요")
#             demo.launch(share=True, server_port=7861, debug=False)
            
#         elif choice == "3":
#             print("🏠 기본 설정으로 실행합니다...")
#             demo.launch()
            
#         else:  # 기본값: choice == "1" 또는 빈 값
#             print("🏠 로컬에서만 실행합니다...")
#             demo.launch(
#                 share=False,  # 공개 링크 없음
#                 server_port=7861,
#                 debug=True,
#                 show_tips=False,
#                 quiet=False
#             )
            
#     except KeyboardInterrupt:
#         print("\n\n👋 사용자가 프로그램을 종료했습니다.")
        
#     except Exception as e:
#         print(f"\n❌ 실행 중 오류 발생: {str(e)}")
#         print("\n🔧 문제 해결 방법:")
#         print("1. 포트가 이미 사용 중일 수 있습니다. 다른 포트를 시도해보세요.")
#         print("2. 방화벽이 차단하고 있을 수 있습니다.")
#         print("3. 가장 간단한 실행: demo.launch()")
        
#         # 간단한 백업 실행
#         try:
#             print("\n🔄 간단한 모드로 재시도...")
#             demo.launch()
#         except:
#             print("❌ 백업 실행도 실패했습니다.")

# # 실행 코드 교체
# if 'demo' in locals():
#     safe_launch_demo(demo)
# else:
#     print("❌ demo 객체가 정의되지 않았습니다. 먼저 Gradio 인터페이스를 생성해주세요.")

# # 또는 직접 실행하는 경우:
# # demo.launch(share=False, server_port=7862)  # 다른 포트 사용

In [ ]:
# import gradio as gr
# from langchain_core.messages import HumanMessage, AIMessage
# import tempfile
# import os
# import shutil

# def process_uploaded_file(file_obj):
#     """업로드된 파일 처리 - Gradio 호환"""
#     if not file_obj:
#         return "파일이 업로드되지 않았습니다."
    
#     try:
#         global full_text, chunks, index, embedding_model, metadatas, pdf_file
        
#         # 파일 객체에서 경로 추출
#         if hasattr(file_obj, 'name'):
#             file_path = file_obj.name
#         else:
#             file_path = str(file_obj)
        
#         # PDF 파일 처리
#         if file_path.endswith('.pdf'):
#             pdf_file = file_path
#             full_text = extract_text_with_fitz(file_path)
            
#             if full_text:
#                 chunks = chunk_text(full_text)
#                 if chunks:
#                     index, embedding_model, chunks, metadatas = load_or_create_vectorstore(
#                         chunks, pdf_file, save_name="uploaded_file_vectors"
#                     )
#                     return f"✅ 파일 '{os.path.basename(file_path)}'이 성공적으로 처리되었습니다!\n총 {len(chunks)}개 청크로 분할되었습니다.\n\n이제 문서에 대해 질문해보세요!"
#                 else:
#                     return "❌ 파일에서 텍스트를 추출할 수 없었습니다."
#             else:
#                 return "❌ 파일 처리에 실패했습니다."
#         else:
#             return "❌ 현재 PDF 파일만 지원됩니다."
            
#     except Exception as e:
#         return f"파일 처리 중 오류 발생: {str(e)}"

# def run_safe_agent(user_query: str) -> str:
#     """안전한 에이전트 실행"""
#     try:
#         # 문서 정보 요청
#         if "문서" in user_query and any(word in user_query for word in ["정보", "상태", "알려"]):
#             if 'chunks' in globals() and chunks:
#                 total_chars = sum(len(chunk) for chunk in chunks)
#                 return f"""
# 📄 현재 로드된 문서 정보:
# - 파일명: {pdf_file if 'pdf_file' in globals() else '알 수 없음'}
# - 총 청크 수: {len(chunks)}
# - 총 글자 수: {total_chars:,}
# - 평균 청크 크기: {total_chars // len(chunks):,} 글자

# 🛠️ 사용 가능한 기능:
# - 문서 검색: "교통약자에 대해 찾아줘"
# - 웹 검색: "최신 AI 뉴스 검색해줘"  
# - 내용 요약: "이 문서를 요약해줘"
# """
#             else:
#                 return "로드된 문서가 없습니다. PDF 파일을 먼저 업로드해주세요."
        
#         # 문서 검색
#         elif any(keyword in user_query for keyword in ["교통", "고령", "약자", "정책", "검색", "찾아"]):
#             if 'index' in globals() and index is not None:
#                 results = search_documents_openai(user_query, index, embedding_model, chunks, metadatas, k=3)
#                 if results:
#                     response = f"🔍 '{user_query}' 검색 결과:\n\n"
#                     for i, result in enumerate(results, 1):
#                         response += f"**결과 {i}** (유사도: {result['score']:.3f})\n"
#                         response += f"{result['content'][:300]}{'...' if len(result['content']) > 300 else ''}\n\n"
#                     return response
#                 else:
#                     return f"'{user_query}'에 대한 검색 결과가 없습니다."
#             else:
#                 return "문서가 업로드되지 않았습니다. 먼저 PDF 파일을 업로드해주세요."
        
#         # 요약 요청
#         elif "요약" in user_query:
#             if 'chunks' in globals() and chunks:
#                 # 전체 문서의 처음 몇 개 청크를 요약
#                 sample_content = "\n".join(chunks[:5])
#                 llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)
#                 response = llm.invoke([
#                     HumanMessage(content=f"다음 내용을 자연스럽게 요약해주세요:\n\n{sample_content}")
#                 ])
#                 return f"📝 문서 요약:\n\n{response.content}"
#             else:
#                 return "요약할 문서가 없습니다. 먼저 PDF 파일을 업로드해주세요."
        
#         # 웹 검색
#         elif any(keyword in user_query for keyword in ["최신", "뉴스", "날씨", "현재"]):
#             try:
#                 search_tool = DuckDuckGoSearchRun()
#                 results = search_tool.run(user_query)
#                 return f"🌐 웹 검색 결과:\n\n{results}"
#             except Exception as e:
#                 return f"웹 검색 중 오류 발생: {str(e)}"
        
#         # 기본 대화
#         else:
#             llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
#             response = llm.invoke([HumanMessage(content=user_query)])
#             return response.content
        
#     except Exception as e:
#         return f"죄송합니다. 처리 중 오류가 발생했습니다: {str(e)}\n\n다른 질문을 시도해보시거나, 문서를 먼저 업로드해주세요."

# def chat_with_agent(message, history):
#     """에이전트와 채팅"""
#     if not message.strip():
#         return history, ""
    
#     bot_response = run_safe_agent(message)
#     history.append([message, bot_response])
#     return history, ""

# # Gradio 인터페이스 - 수정된 버전
# with gr.Blocks(title="🤖 멀티모달 AI 어시스턴트", theme="soft") as demo:
    
#     gr.Markdown("""
#     # 🤖 멀티모달 AI 어시스턴트
    
#     **기능:**
#     - 📁 PDF 파일 업로드 및 분석
#     - 🔍 업로드된 문서 검색 및 요약
#     - 🌐 웹 검색으로 최신 정보 조회
#     - 💬 자연어 대화형 인터페이스
#     """)
    
#     with gr.Row():
#         # 좌측: 파일 업로드 영역
#         with gr.Column(scale=1):
#             gr.Markdown("### 📁 파일 업로드")
            
#             # 수정된 부분: type 매개변수 제거
#             file_upload = gr.File(
#                 label="PDF 파일 선택",
#                 file_types=[".pdf"]
#             )
            
#             file_status = gr.Textbox(
#                 label="파일 처리 상태",
#                 value="파일을 업로드해주세요...",
#                 interactive=False,
#                 lines=3
#             )
            
#             gr.Markdown("""
#             ### 💡 사용 팁
#             **예시 질문:**
#             - "이 문서를 요약해줘"
#             - "교통약자에 대해 찾아줘"
#             - "고령자 정책은 뭐야?"
#             - "최신 AI 뉴스 검색해줘"
#             - "문서 정보 알려줘"
#             """)
        
#         # 우측: 채팅 영역  
#         with gr.Column(scale=2):
#             gr.Markdown("### 💬 AI 어시스턴트와 대화")
            
#             chatbot = gr.Chatbot(
#                 value=[],
#                 height=400,
#                 show_label=False
#             )
            
#             with gr.Row():
#                 msg_input = gr.Textbox(
#                     placeholder="질문을 입력하세요... (예: 이 문서를 요약해줘)",
#                     show_label=False,
#                     scale=4
#                 )
#                 send_btn = gr.Button("전송", variant="primary", scale=1)
            
#             # 예제 버튼들
#             with gr.Row():
#                 example_btns = [
#                     gr.Button("📄 문서 정보", size="sm"),
#                     gr.Button("📝 문서 요약", size="sm"),
#                     gr.Button("🔍 내용 검색", size="sm"),
#                     gr.Button("🌐 웹 검색", size="sm")
#                 ]
    
#     # 이벤트 핸들러들
    
#     # 파일 업로드 처리
#     file_upload.change(
#         process_uploaded_file,
#         inputs=[file_upload],
#         outputs=[file_status]
#     )
    
#     # 채팅 처리
#     msg_input.submit(
#         chat_with_agent,
#         inputs=[msg_input, chatbot],
#         outputs=[chatbot, msg_input]
#     )
    
#     send_btn.click(
#         chat_with_agent,
#         inputs=[msg_input, chatbot],
#         outputs=[chatbot, msg_input]
#     )
    
#     # 예제 버튼 클릭 처리
#     def click_example(example_text, history):
#         return chat_with_agent(example_text, history)
    
#     example_btns[0].click(
#         lambda hist: click_example("문서 정보를 알려줘", hist),
#         inputs=[chatbot],
#         outputs=[chatbot, msg_input]
#     )
    
#     example_btns[1].click(
#         lambda hist: click_example("이 문서를 요약해줘", hist),
#         inputs=[chatbot],
#         outputs=[chatbot, msg_input]
#     )
    
#     example_btns[2].click(
#         lambda hist: click_example("교통약자에 대해 검색해줘", hist),
#         inputs=[chatbot],
#         outputs=[chatbot, msg_input]
#     )
    
#     example_btns[3].click(
#         lambda hist: click_example("최신 AI 뉴스를 검색해줘", hist),
#         inputs=[chatbot],
#         outputs=[chatbot, msg_input]
#     )

# # 실행
# print("🚀 멀티모달 AI 어시스턴트 시작!")
# print("- 파일 업로드 ✅")
# print("- 문서 검색 ✅") 
# print("- 웹 검색 ✅")
# print("- 요약 기능 ✅")
# demo.launch(share=True, server_port=7860, debug=True)

In [ ]:
# import gradio as gr
# from langchain_core.messages import HumanMessage, AIMessage
# import tempfile
# import os

# def process_uploaded_file(file_path):
#     """업로드된 파일 처리"""
#     if not file_path:
#         return "파일이 업로드되지 않았습니다."
    
#     try:
#         global full_text, chunks, index, embedding_model, metadatas, pdf_file
        
#         # PDF 파일 처리
#         if file_path.endswith('.pdf'):
#             pdf_file = file_path
#             full_text = extract_text_with_fitz(file_path)
            
#             if full_text:
#                 chunks = chunk_text(full_text)
#                 if chunks:
#                     index, embedding_model, chunks, metadatas = load_or_create_vectorstore(
#                         chunks, pdf_file, save_name="uploaded_file_vectors"
#                     )
#                     return f"✅ 파일 '{os.path.basename(file_path)}'이 성공적으로 처리되었습니다!\n총 {len(chunks)}개 청크로 분할되었습니다."
#                 else:
#                     return "❌ 파일에서 텍스트를 추출할 수 없었습니다."
#             else:
#                 return "❌ 파일 처리에 실패했습니다."
#         else:
#             return "❌ 현재 PDF 파일만 지원됩니다."
            
#     except Exception as e:
#         return f"파일 처리 중 오류 발생: {str(e)}"

# def chat_with_agent(message, history):
#     """에이전트와 채팅"""
#     try:
#         if not message.strip():
#             return history, ""
        
#         # 초기 상태 설정
#         initial_state = {
#             "messages": [HumanMessage(content=message)],
#             "user_query": message,
#             "search_results": [],
#             "summary": "",
#             "tool_calls": []
#         }
        
#         # 에이전트 실행
#         result = graph.invoke(initial_state)
#         final_message = result["messages"][-1]
        
#         # 응답 추출
#         if hasattr(final_message, 'content'):
#             bot_response = run_safe_agent(message)
#         else:
#             bot_response = str(final_message)
        
#         # 히스토리에 추가
#         history.append([message, bot_response])
#         return history, ""
        
#     except Exception as e:
#         error_response = f"죄송합니다. 오류가 발생했습니다: {str(e)}"
#         history.append([message, error_response])
#         return history, ""

# # Gradio 인터페이스
# with gr.Blocks(title="🤖 멀티모달 AI 어시스턴트", theme="soft") as demo:
    
#     gr.Markdown("""
#     # 🤖 멀티모달 AI 어시스턴트
    
#     **기능:**
#     - 📁 PDF 파일 업로드 및 분석
#     - 🔍 업로드된 문서 검색 및 요약
#     - 🌐 웹 검색으로 최신 정보 조회
#     - 💬 자연어 대화형 인터페이스
#     """)
    
#     with gr.Row():
#         # 좌측: 파일 업로드 영역
#         with gr.Column(scale=1):
#             gr.Markdown("### 📁 파일 업로드")
            
#             file_upload = gr.File(
#                 label="PDF 파일 선택",
#                 file_types=[".pdf"],
#                 type="filepath"
#             )
            
#             file_status = gr.Textbox(
#                 label="파일 처리 상태",
#                 value="파일을 업로드해주세요...",
#                 interactive=False,
#                 lines=3
#             )
            
#             gr.Markdown("""
#             ### 💡 사용 팁
#             **예시 질문:**
#             - "이 문서를 요약해줘"
#             - "교통약자에 대해 찾아줘"
#             - "고령자 정책은 뭐야?"
#             - "최신 AI 뉴스 검색해줘"
#             - "문서 정보 알려줘"
#             """)
        
#         # 우측: 채팅 영역  
#         with gr.Column(scale=2):
#             gr.Markdown("### 💬 AI 어시스턴트와 대화")
            
#             chatbot = gr.Chatbot(
#                 value=[],
#                 height=400,
#                 show_label=False
#             )
            
#             with gr.Row():
#                 msg_input = gr.Textbox(
#                     placeholder="질문을 입력하세요... (예: 이 문서를 요약해줘)",
#                     show_label=False,
#                     scale=4
#                 )
#                 send_btn = gr.Button("전송", variant="primary", scale=1)
            
#             # 예제 버튼들
#             with gr.Row():
#                 example_btns = [
#                     gr.Button("📄 문서 정보", size="sm"),
#                     gr.Button("📝 문서 요약", size="sm"),
#                     gr.Button("🔍 내용 검색", size="sm"),
#                     gr.Button("🌐 웹 검색", size="sm")
#                 ]
    
#     # 이벤트 핸들러들
    
#     # 파일 업로드 처리
#     def handle_file_upload(file_path):
#         if file_path:
#             result = process_uploaded_file(file_path)
#             return result
#         return "파일을 선택해주세요."
    
#     file_upload.change(
#         handle_file_upload,
#         inputs=[file_upload],
#         outputs=[file_status]
#     )
    
#     # 채팅 처리
#     def handle_chat(message, history):
#         return chat_with_agent(message, history)
    
#     msg_input.submit(
#         handle_chat,
#         inputs=[msg_input, chatbot],
#         outputs=[chatbot, msg_input]
#     )
    
#     send_btn.click(
#         handle_chat,
#         inputs=[msg_input, chatbot],
#         outputs=[chatbot, msg_input]
#     )
    
#     # 예제 버튼 클릭 처리
#     def click_example(example_text, history):
#         return chat_with_agent(example_text, history)
    
#     example_btns[0].click(
#         lambda hist: click_example("문서 정보를 알려줘", hist),
#         inputs=[chatbot],
#         outputs=[chatbot, msg_input]
#     )
    
#     example_btns[1].click(
#         lambda hist: click_example("이 문서를 요약해줘", hist),
#         inputs=[chatbot],
#         outputs=[chatbot, msg_input]
#     )
    
#     example_btns[2].click(
#         lambda hist: click_example("교통약자에 대해 검색해줘", hist),
#         inputs=[chatbot],
#         outputs=[chatbot, msg_input]
#     )
    
#     example_btns[3].click(
#         lambda hist: click_example("최신 AI 뉴스를 검색해줘", hist),
#         inputs=[chatbot],
#         outputs=[chatbot, msg_input]
#     )

# # 실행
# if 'graph' in locals():
#     print("🚀 멀티모달 AI 어시스턴트 시작!")
#     print("- 파일 업로드 ✅")
#     print("- 문서 검색 ✅") 
#     print("- 웹 검색 ✅")
#     print("- 요약 기능 ✅")
#     demo.launch(share=True, server_port=7860, debug=True)
# else:
#     print("❌ 그래프가 준비되지 않았습니다.")
#     print("앞의 셀들을 먼저 실행해주세요.")

In [ ]:
# import gradio as gr
# import traceback

# def super_safe_chat(message, history):
#     """완전히 안전한 채팅 함수"""
#     try:
#         print(f"[DEBUG] 받은 메시지: '{message}'")
        
#         # 빈 메시지 처리
#         if not message or not message.strip():
#             print("[DEBUG] 빈 메시지")
#             return history, ""
        
#         # 기본 응답들 (에러 없이 확실히 작동)
#         message_lower = message.lower().strip()
        
#         if "안녕" in message:
#             response = "안녕하세요! 저는 AI 어시스턴트입니다. 😊"
#         elif "테스트" in message:
#             response = "테스트 성공! 기본 기능이 정상 작동합니다. ✅"
#         elif "도움" in message or "help" in message_lower:
#             response = """도움말:
# - 기본 대화: 안녕, 테스트 등
# - 파일 업로드 후 문서 질문 가능
# - 현재는 안전 모드로 실행 중입니다."""
#         else:
#             response = f"메시지 '{message}'를 받았습니다. 현재 안전 모드에서 실행 중입니다."
        
#         print(f"[DEBUG] 응답 생성: '{response}'")
        
#         # 히스토리에 추가
#         new_history = history + [[message, response]]
#         print(f"[DEBUG] 히스토리 업데이트 완료")
        
#         return new_history, ""
        
#     except Exception as e:
#         # 에러 로깅
#         error_msg = f"채팅 처리 오류: {str(e)}"
#         print(f"[ERROR] {error_msg}")
#         print(f"[ERROR] 상세: {traceback.format_exc()}")
        
#         # 안전한 에러 응답
#         safe_response = f"죄송합니다. 처리 중 문제가 발생했습니다. (에러: {str(e)[:50]})"
        
#         try:
#             return history + [[message, safe_response]], ""
#         except:
#             # 최후의 수단
#             return [["시스템", "에러가 발생했습니다. 페이지를 새로고침해주세요."]], ""

# def safe_file_handler(file_obj):
#     """안전한 파일 처리 함수"""
#     try:
#         if not file_obj:
#             return "파일이 선택되지 않았습니다."
        
#         # 파일 정보만 표시 (실제 처리는 안전을 위해 생략)
#         file_name = getattr(file_obj, 'name', str(file_obj))
#         return f"✅ 파일이 업로드되었습니다: {file_name}\n현재 안전 모드에서는 파일 처리가 제한됩니다."
        
#     except Exception as e:
#         print(f"[ERROR] 파일 처리 오류: {str(e)}")
#         return f"파일 처리 중 오류 발생: {str(e)}"

# # 안전한 Gradio 인터페이스
# with gr.Blocks(title="🛡️ 안전 모드 - AI 어시스턴트") as safe_demo:
    
#     gr.Markdown("""
#     # 🛡️ 안전 모드 - AI 어시스턴트
    
#     **현재 안전 모드로 실행 중입니다.**
#     - 기본 대화 기능만 활성화됨
#     - 에러 발생 시 안전한 응답 제공
#     - 디버깅 정보가 콘솔에 출력됨
    
#     **테스트 메시지:** 안녕, 테스트, 도움말
#     """)
    
#     with gr.Row():
#         with gr.Column(scale=1):
#             gr.Markdown("### 📁 파일 업로드 (안전 모드)")
#             file_upload = gr.File(label="파일 선택", type="file")
#             file_status = gr.Textbox(
#                 label="파일 상태", 
#                 value="안전 모드: 파일 업로드 테스트만 가능",
#                 lines=3
#             )
            
#             gr.Markdown("""
#             ### 🧪 테스트 명령어
#             - **안녕** → 인사 응답
#             - **테스트** → 기능 확인
#             - **도움말** → 사용법 안내
#             """)
        
#         with gr.Column(scale=2):
#             gr.Markdown("### 💬 안전 모드 채팅")
            
#             chatbot = gr.Chatbot(
#                 value=[["시스템", "안전 모드로 시작되었습니다. '안녕'이라고 말해보세요!"]],
#                 height=400
#             )
            
#             with gr.Row():
#                 msg_input = gr.Textbox(
#                     placeholder="메시지를 입력하세요 (예: 안녕, 테스트)",
#                     scale=4,
#                     show_label=False
#                 )
#                 send_btn = gr.Button("전송", variant="primary", scale=1)
            
#             # 빠른 테스트 버튼들
#             with gr.Row():
#                 test_buttons = [
#                     gr.Button("안녕", size="sm"),
#                     gr.Button("테스트", size="sm"), 
#                     gr.Button("도움말", size="sm"),
#                     gr.Button("초기화", size="sm", variant="secondary")
#                 ]

#     # 이벤트 연결
#     file_upload.change(safe_file_handler, [file_upload], [file_status])
    
#     msg_input.submit(super_safe_chat, [msg_input, chatbot], [chatbot, msg_input])
#     send_btn.click(super_safe_chat, [msg_input, chatbot], [chatbot, msg_input])
    
#     # 테스트 버튼 이벤트
#     test_buttons[0].click(lambda hist: super_safe_chat("안녕", hist), [chatbot], [chatbot, msg_input])
#     test_buttons[1].click(lambda hist: super_safe_chat("테스트", hist), [chatbot], [chatbot, msg_input])
#     test_buttons[2].click(lambda hist: super_safe_chat("도움말", hist), [chatbot], [chatbot, msg_input])
#     test_buttons[3].click(lambda: ([["시스템", "채팅이 초기화되었습니다."]], ""), [], [chatbot, msg_input])

# print("🛡️ 안전 모드 실행...")
# print("🔍 모든 디버깅 정보가 콘솔에 출력됩니다.")
# safe_demo.launch(share=False, server_port=7863, debug=True)

c:\AI_Prompt\workspace\ai_agent_work2\langgraph_uv\.venv\Lib\site-packages\gradio\deprecation.py:40: UserWarning: `height` is deprecated in `Interface()`, please use it within `launch()` instead.
  warnings.warn(value)
c:\AI_Prompt\workspace\ai_agent_work2\langgraph_uv\.venv\Lib\site-packages\gradio\deprecation.py:43: UserWarning: You have unused kwarg parameters in Textbox, please remove them: {'scale': 4}
  warnings.warn(
c:\AI_Prompt\workspace\ai_agent_work2\langgraph_uv\.venv\Lib\site-packages\gradio\deprecation.py:43: UserWarning: You have unused kwarg parameters in Button, please remove them: {'scale': 1}
  warnings.warn(
c:\AI_Prompt\workspace\ai_agent_work2\langgraph_uv\.venv\Lib\site-packages\gradio\deprecation.py:43: UserWarning: You have unused kwarg parameters in Button, please remove them: {'size': 'sm'}
  warnings.warn(


🛡️ 안전 모드 실행...
🔍 모든 디버깅 정보가 콘솔에 출력됩니다.
Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


In [ ]:
demo.close()

NameError: name 'demo' is not defined